In [1]:
from hct.envs.low_level_env import LowLevelEnv
from hct.envs.ant import Ant

import functools
import jax
import os

from datetime import datetime
from jax import numpy as jp
import matplotlib.pyplot as plt

from IPython.display import HTML, clear_output

import brax

import flax
from brax import envs
from brax.io import model
from brax.io import json
from brax.io import html
from hct import train as ppo
from brax.training.agents.sac import train as sac
from brax.base import Transform, Motion
from brax.math import euler_to_quat
from absl import logging

import getpass
import socket

from brax.kinematics import world_to_joint, inverse, forward


env = LowLevelEnv()

jit_env_reset = jax.jit(env.reset)

rng = jax.random.PRNGKey(seed=1)

state = jit_env_reset(rng=jax.random.PRNGKey(seed=0))


rollout = []

state = jit_env_reset(rng=rng)
for _ in range(100):
  rollout.append(state.pipeline_state)
  rng, rng1 = jax.random.split(rng)
  state = jit_env_reset(rng1)
display = HTML(html.render(env.sys.replace(dt=env.dt), rollout))


2023-06-22 04:08:51.635298: E external/xla/xla/stream_executor/cuda/cuda_driver.cc:268] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)


In [2]:
print(html.render(env.sys.replace(dt=env.dt), rollout))

<!DOCTYPE html>
<html>

  <head>
    <title>Brax visualizer</title>
    <link rel="shortcut icon" type="image/x-icon" href="/favicon.ico">
  </head>

  <style>
    body {
      margin: 0;
      padding: 0;
    }

    #brax-viewer {
      height: 480px;
      margin: 0;
      padding: 0;
    }
  </style>
  <script async src="https://unpkg.com/es-module-shims@1.6.3/dist/es-module-shims.js"></script>

  <script type="importmap">
    {
      "imports": {
        "three": "https://unpkg.com/three@0.150.1/build/three.module.js",
        "three/addons/": "https://unpkg.com/three@0.150.1/examples/jsm/",
        "lilgui": "https://cdn.jsdelivr.net/npm/lil-gui@0.18.0/+esm",
        "viewer": "https://cdn.jsdelivr.net/gh/google/brax@v0.9.1/brax/visualizer/js/viewer.js"
      }
    }
  </script>

  <script type="application/javascript">
  var system = {"dt": 0.04999999701976776, "gravity": [0.0, 0.0, -9.8100004196167], "viscosity": 0.0, "density": 0.0, "link": {"transform": {"pos": [[0.0, 0.0, 0.0

In [14]:

for i in rollout:
    print(i.q[7:])
    print(env.observer.goal_q_limit[6:,:])

[-0.26735234  0.76515913 -0.09382463 -0.72502077 -0.48836255 -0.7125042
  0.32839948  0.8497935 ]
[[-0.5235988  0.5235988]
 [ 0.5235988  1.2217305]
 [-0.5235988  0.5235988]
 [-1.2217305 -0.5235988]
 [-0.5235988  0.5235988]
 [-1.2217305 -0.5235988]
 [-0.5235988  0.5235988]
 [ 0.5235988  1.2217305]]
[-0.45535922  1.0223168  -0.41956162 -1.2065278  -0.09744    -0.5317789
  0.19062322  0.9119594 ]
[[-0.5235988  0.5235988]
 [ 0.5235988  1.2217305]
 [-0.5235988  0.5235988]
 [-1.2217305 -0.5235988]
 [-0.5235988  0.5235988]
 [-1.2217305 -0.5235988]
 [-0.5235988  0.5235988]
 [ 0.5235988  1.2217305]]
[ 0.24317878  1.1772184  -0.47865507 -0.7233528   0.09459734 -1.0363222
 -0.5207428   1.1694951 ]
[[-0.5235988  0.5235988]
 [ 0.5235988  1.2217305]
 [-0.5235988  0.5235988]
 [-1.2217305 -0.5235988]
 [-0.5235988  0.5235988]
 [-1.2217305 -0.5235988]
 [-0.5235988  0.5235988]
 [ 0.5235988  1.2217305]]
[ 0.16727531  0.9659068  -0.02418718 -0.6199612   0.10304701 -1.0375216
  0.41076845  1.0944686 ]
[[-0.

In [18]:
def sample_goal(env, rng: jp.ndarray, state):
    """
    Samples normalised goal and outputs a goal state 

    Args:
      rng: jp.ndarray

    Returns:
      goal: Goal
    """
    def sample(carry):
      rng, _ = carry
      rng, rng1 = jax.random.split(rng)
      g = jax.random.uniform(
          rng1, 
          shape=env.observer.goal_size, 
          minval=-1, 
          maxval=1
        )
      goal = env.goalconstructor.create_goal(g, state)
      return rng, goal
    def reject(carry):
      _, goal = carry
      z = goal.x.pos[:,2]
      polar = goal.bpg[6]
      cond = jp.any(z < env.observer.goal_z_cond) | (polar > env.observer.goal_polar_cond)
      return cond
    init_g = jax.random.uniform(
        rng, 
        shape=env.observer.goal_size, 
        minval=-1, 
        maxval=1
    )
    init_val = rng, env.goalconstructor.create_goal(init_g, state)
    goal = jax.lax.while_loop(reject, sample, init_val)[1]
    return goal

print(sample_goal(env, rng, state.pipeline_state).x)
goal = state.info['goal']
z = goal.x.pos[:,2]
polar = goal.bpg[6]
print(jp.any(z < env.observer.goal_z_cond) & (polar > env.observer.goal_polar_cond))

Transform(pos=Array([[ 4.488135  ,  0.8433745 ,  0.7385601 ],
       [ 5.0284357 ,  0.975194  ,  0.737318  ],
       [ 5.340225  ,  0.91163635,  0.26961306],
       [ 4.205926  ,  1.3212267 ,  0.64667207],
       [ 3.9710238 ,  1.5930551 ,  0.20971629],
       [ 3.9851441 ,  0.58724827,  0.76229775],
       [ 3.7129018 ,  0.34470505,  0.32979694],
       [ 4.6203933 ,  0.3112268 ,  0.8337907 ],
       [ 4.6555243 , -0.03332058,  0.38651794]], dtype=float32), rot=Array([[ 0.98002607, -0.07885338, -0.03403434, -0.17936778],
       [ 0.9307127 , -0.07130166, -0.04787759, -0.35552466],
       [ 0.8031066 , -0.261516  ,  0.40409687, -0.3511908 ],
       [ 0.99349153, -0.08200782, -0.02551289, -0.07482287],
       [ 0.90145254, -0.34537894, -0.25493622, -0.05571675],
       [ 0.9879139 , -0.0804901 , -0.02995889, -0.12903474],
       [ 0.89741915,  0.20266528, -0.38366124, -0.07981028],
       [ 0.93207663, -0.07148574, -0.0476023 , -0.35193312],
       [ 0.8423064 ,  0.40636343,  0.17048633

In [3]:
env = LowLevelEnv()

jit_env_reset = jax.jit(env.reset)

rng = jax.random.PRNGKey(seed=1)

state = jit_env_reset(rng=jax.random.PRNGKey(seed=0))
HTML(html.render(env.sys, [state.pipeline_state]))


In [8]:
import jaxlib
print(type(state.obs)==jaxlib.xla_extension.ArrayImpl)
type(jp.array([2]))


True


jaxlib.xla_extension.ArrayImpl

In [12]:
from brax.base import Transform
from brax.kinematics import forward
from flax import struct
from typing import Optional
rng = jax.random.PRNGKey(0)
qd = jax.random.normal(rng, shape = (env.sys.qd_size(),))
q = jp.zeros(env.sys.q_size()).at[3].set(1)
#print(qd)
x, xd = forward(env.sys, q, qd)
x.__mul__(jp.zeros((9,7)).at[0].set(1))


TypeError: mul got incompatible shapes for broadcasting: (9, 3), (9, 7).

In [11]:
import jax
import jax.numpy as jp
import os
from hct.envs.env_tools import EnvTools
p = os.getcwd()
b = 2

g = jp.array([-0.2,0.5,0.6])
g_limit = jp.array([[5.0,-5.0],[5.0,-5.0],[5.0,-5.0]])
print(g_limit[:,0].shape)

class A(EnvTools):
    @staticmethod
    def unnormalize(
            normalized_value, 
            min_value, 
            max_value, 
            a=-1, 
            b=1
        ):
        if max_value == min_value:
            return 0
        value = min_value + ((normalized_value - a) * (max_value - min_value)) / (b - a)
        return value
    
    def backproj(self, g, g_min, g_max):
        """Backprojects goal from normalised values to unnormalised values"""
        bpg = self.\
            vmap(in_axes=(0, 0, 0, None, None))\
                .unnormalize(g, min_value=g_min, max_value=g_max, a=-1, b=1)
        return bpg

a=A(env.sys)
#a.backproj(g, g_limit[:,0], g_limit[:,1])


def unnormalize(
          normalized_value, 
          min_value, 
          max_value, 
          a=-1, 
          b=1
    ):
        def func(normalized_value):
            return min_value + ((normalized_value - a) * (max_value - min_value)) / (b - a)
        def zero(normalized_value):
            return 0.0
        value = jax.lax.cond(max_value == min_value, func, zero, normalized_value)
        
        return value
jax.vmap(unnormalize, in_axes = (0, 0, 0, None, None))(g, g, g, -1, 1)



(3,)


TypeError: select cases must have the same shapes, got [(2,), (1,)].

In [ ]:
from brax.base import Transform
def world_to_egocentric(
        t: jp.ndarray, 
        x:jp.ndarray
) -> Transform:
    """Converts a transform in world frame to transform in egocentric frame.

    Args:
        t: transform in world frame
        x: world frame transform of agent

    Returns:
        t': transform in egocentric frame
    """
    rpos = x.pos[0]
    rrot = x.rot[0]
    rx = Transform.create(rpos,rrot)
    return t.vmap(in_axes=(0,None)).to_local(rx)

def egocentric_to_world(
        t: jp.ndarray, 
        x: jp.ndarray
) -> Transform:
    """Converts a transform in world frame to transform in egocentric frame.

    Args:
        t: transform in egocentric frame
        x: world frame transform of agent

    Returns:
        t': transform in world frame
    """
    rpos = x.pos[0]
    rrot = x.rot[0]
    rx = Transform.create(rpos,rrot)
    return rx.vmap(in_axes=(None,0)).do(t)
x = state.pipeline_state.x
print(x)
t = world_to_egocentric(x,x)
x = egocentric_to_world(t,x)
print(t)
print(x)

Transform(pos=Array([[ 0.        ,  0.        ,  0.55      ],
       [ 0.4       ,  0.4       ,  0.55      ],
       [ 0.61612093,  0.61612093,  0.07399213],
       [-0.4       ,  0.4       ,  0.55      ],
       [-0.61612093,  0.61612093,  0.07399213],
       [-0.4       , -0.4       ,  0.55      ],
       [-0.61612093, -0.61612093,  0.07399213],
       [ 0.4       , -0.4       ,  0.55      ],
       [ 0.7464102 , -0.7464102 ,  0.2671573 ]], dtype=float32), rot=Array([[ 1.        ,  0.        ,  0.        ,  0.        ],
       [ 1.        ,  0.        ,  0.        ,  0.        ],
       [ 0.87758255, -0.33900505,  0.33900505,  0.        ],
       [ 1.        ,  0.        ,  0.        ,  0.        ],
       [ 0.87758255, -0.33900505, -0.33900505,  0.        ],
       [ 1.        ,  0.        ,  0.        ,  0.        ],
       [ 0.87758255,  0.33900505, -0.33900505,  0.        ],
       [ 1.        ,  0.        ,  0.        ,  0.        ],
       [ 0.9659259 ,  0.18301271,  0.18301271

In [ ]:
def x_world_to_egocentric(x: jp.ndarray):
    rpos = x.pos[0]
    rrot = x.rot[0]
    rx = Transform.create(rpos,rrot)
    return x.vmap(in_axes=(0,None)).to_local(rx)

x2 = x_world_to_egocentric(x)
j2, jd2, _, _  = world_to_joint(env.sys, x2, xd)
print(j2)

Transform(pos=Array([[ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00],
       [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00],
       [ 2.9802322e-08,  2.9802322e-08,  0.0000000e+00],
       [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00],
       [-2.9802322e-08,  2.9802322e-08,  0.0000000e+00],
       [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00],
       [-2.9802322e-08, -2.9802322e-08,  0.0000000e+00],
       [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00],
       [ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00]], dtype=float32), rot=Array([[ 1.        ,  0.        ,  0.        ,  0.        ],
       [ 1.        ,  0.        ,  0.        ,  0.        ],
       [ 0.87758255, -0.33900505,  0.33900505,  0.        ],
       [ 1.        ,  0.        ,  0.        ,  0.        ],
       [ 0.87758255, -0.33900505, -0.33900505,  0.        ],
       [ 1.        ,  0.        ,  0.        ,  0.        ],
       [ 0.87758255,  0.33900505, -0.33900505,  0.        ],
       [ 1.        ,  0. 

In [ ]:


jit_env_reset = jax.jit(env.reset)
jit_env_step = jax.jit(env.step)
jit_inference_fn = jax.random.uniform
jit_inference_fn = jax.random.choice

rollout = []
rng = jax.random.PRNGKey(seed=1)
state = jit_env_reset(rng=rng)
for _ in range(1000):
  rollout.append(state.pipeline_state)
  act_rng, rng = jax.random.split(rng)
  act = jit_inference_fn(act_rng, a = jp.array([-1,1]), shape=(8,))
  state = jit_env_step(state, act)
h = html.render(env.sys.replace(dt=env.dt), rollout)
display(HTML(h))

In [ ]:
print(h)

<!DOCTYPE html>
<html>

  <head>
    <title>Brax visualizer</title>
    <link rel="shortcut icon" type="image/x-icon" href="/favicon.ico">
  </head>

  <style>
    body {
      margin: 0;
      padding: 0;
    }

    #brax-viewer {
      height: 480px;
      margin: 0;
      padding: 0;
    }
  </style>
  <script async src="https://unpkg.com/es-module-shims@1.6.3/dist/es-module-shims.js"></script>

  <script type="importmap">
    {
      "imports": {
        "three": "https://unpkg.com/three@0.150.1/build/three.module.js",
        "three/addons/": "https://unpkg.com/three@0.150.1/examples/jsm/",
        "lilgui": "https://cdn.jsdelivr.net/npm/lil-gui@0.18.0/+esm",
        "viewer": "https://cdn.jsdelivr.net/gh/google/brax@v0.9.1/brax/visualizer/js/viewer.js"
      }
    }
  </script>

  <script type="application/javascript">
  var system = {"dt": 0.04999999701976776, "gravity": [0.0, 0.0, -9.8100004196167], "viscosity": 0.0, "density": 0.0, "link": {"transform": {"pos": [[0.0, 0.0, 0.0

In [ ]:
pos = state.pipeline_state.x.pos
vel = state.pipeline_state.xd.vel
pos = pos[[2,4,6,8],:]
vel = vel[[2,4,6,8],:]
ee = Transform.create(jp.array(pos))
eevel = Motion.create(jp.array(vel))

x = world_to_joint(env.sys, ee,eevel)


ValueError: vmap got inconsistent sizes for array axes to be mapped:
  * most axes (2 of them) had size 4, e.g. axis 0 of argument self.pos of type float32[4,3];
  * some axes (2 of them) had size 9, e.g. axis 0 of argument o.pos of type float32[9,3]

In [ ]:
jit_env_reset = jax.jit(env.reset)
jit_env_step = jax.jit(env.step)
rollout = []

rng = jax.random.PRNGKey(seed=1)
state = jit_env_reset(rng=rng)
for _ in range(100):
  rollout.append(state.pipeline_state)
  act_rng, rng = jax.random.split(rng)
  a = jax.random.uniform(act_rng, shape=(8,), minval=-1,maxval=1)
  state = jit_env_step(state, a)

HTML(html.render(env.sys.replace(dt=env.dt), rollout))


In [ ]:
HTML(html.render(env.sys, [state.pipeline_state]))

NameError: name 'state' is not defined

In [ ]:
rng = jax.random.PRNGKey(seed=1)
jit_env_reset = jax.jit(env.reset)
state = jit_env_reset(rng=rng)
pipeline_state = state.pipeline_state
print((type(pipeline_state.q)))


<class 'jaxlib.xla_extension.ArrayImpl'>


In [ ]:
obs = env.observer.get_obs(pipeline_state)
print(obs.shape)


(9, 15)


In [ ]:

pipeline_state = state.pipeline_state
'''print(pipeline_state.q)
print(pipeline_state.qd)
print(pipeline_state.contact)'''
"""
print(env.sys.link_types)
print(env.sys.link_names)
print(env.sys.q_size())
print(env.sys.qd_size())
print(env.sys.dof.limit[0])
print(env.sys.dof.limit[1])
print(env.sys.link_parents)"""

#print(env.sys.actuator.q_id)
'''

ja = 2'''
ja = jp.zeros(env.sys.num_links())

def ja_id(id):
    if env.sys.link_types[id] == '1':
        ja = 0

q = pipeline_state.q
q_idx = jp.indices([env.sys.q_size()])
hinge_link_q_idx = env.sys.q_idx('1')
non_hinge_link_q_idx = jp.delete(q_idx ,hinge_link_q_idx)

def process_sequences(arr, bool_indices):
    # Create a boolean mask of the same length as the array
    mask = jp.zeros_like(arr, dtype=bool)
    
    # Set the mask to True at the given indices
    mask = mask.at[bool_indices].set(True)
    
    # Apply the mask to the array, replacing True sequences with 0
    output = jp.where(mask, 0, arr)
    
    # Remove the zeroed sequences (replaced True sequences) from the array
    output = output[output != 0]
    
    return output
print(q_idx)
print(non_hinge_link_q_idx)

[[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14]]
[0 1 2 3 4 5 6]


In [ ]:

print(pipeline_state.x.pos)

NameError: name 'pipeline_state' is not defined